# Daily Challenge — Construire un chatbot avec Transformers et Gradio

Ce notebook montre comment créer un chatbot conversationnel avec la bibliothèque Transformers de Hugging Face et une interface web Gradio. Le modèle utilisé est `facebook/blenderbot-400M-distill`.

## Objectifs
- Charger et utiliser un modèle conversationnel avec `transformers`.
- Maintenir le contexte via la classe `Conversation`.
- Construire une interface interactive avec Gradio.
- Simuler une conversation en temps réel en Python.

## Choix techniques
- Le pipeline conversational simplifie l’inférence et encapsule le modèle et le tokenizer.
- La classe Conversation permet de conserver l’historique des tours de dialogue.
- Gradio ChatInterface fournit une interface prête à l’emploi pour discuter avec le modèle.

## Étape 0 — Installation et import des bibliothèques

Cette cellule importe `transformers` et `gradio`. Si nécessaire, elle installe les dépendances. L’installation peut prendre du temps car les poids du modèle seront téléchargés lors du premier chargement.

In [16]:
def _safe_imports():
    try:
        import transformers  # noqa: F401
        import gradio  # noqa: F401
        return True
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "gradio", "-q"])
        return True

_safe_imports()
from transformers import pipeline
import gradio as gr

import transformers as _tf
print("Transformers version:", _tf.__version__)
print("Gradio version:", gr.__version__)

Transformers version: 4.55.2
Gradio version: 5.42.0


## Étape 1 — Charger le pipeline conversationnel

Nous chargeons un pipeline de type `conversational` avec le modèle `facebook/blenderbot-400M-distill`. Ce pipeline prend des objets `Conversation` en entrée et les enrichit avec les réponses générées.

In [18]:
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

MODEL_NAME = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(MODEL_NAME)
model = BlenderbotForConditionalGeneration.from_pretrained(MODEL_NAME)
print("Model and Tokenizer loaded:", MODEL_NAME)

Model and Tokenizer loaded: facebook/blenderbot-400M-distill


## Étape 2 — Démarrer et étendre une conversation (console)

Nous créons une instance `Conversation` avec un message utilisateur initial, puis nous la passons au pipeline pour générer une réponse. Nous ajoutons ensuite un nouvel input utilisateur et relançons le pipeline. L’objet `Conversation` conserve l’historique.

In [20]:
# Initialize chat history as an empty list of tuples
chat_history = []

# First turn
user_input_1 = "Hi, I am testing a BlenderBot chatbot. How are you today?"
response_1 = generate_response(user_input_1, chat_history)
print(f"Tour 1:\nUser: {user_input_1}\nBot: {response_1}")
chat_history.append([user_input_1, response_1])

# Second turn
user_input_2 = "What can you do? Give me a very short answer."
response_2 = generate_response(user_input_2, chat_history)
print(f"\nTour 2:\nUser: {user_input_2}\nBot: {response_2}")
chat_history.append([user_input_2, response_2])

print("\nFull chat history:", chat_history)

Tour 1:
User: Hi, I am testing a BlenderBot chatbot. How are you today?
Bot:  I am doing well, how are you? I have never heard of that, what is it?

Tour 2:
User: What can you do? Give me a very short answer.
Bot:  I'm doing great, thanks for asking. It's a chat app that allows you to talk to people over the internet.

Full chat history: [['Hi, I am testing a BlenderBot chatbot. How are you today?', ' I am doing well, how are you? I have never heard of that, what is it?'], ['What can you do? Give me a very short answer.', " I'm doing great, thanks for asking. It's a chat app that allows you to talk to people over the internet."]]


## Étape 3 — Définir la fonction `mini_chatbot` pour Gradio

La fonction prend le `message` utilisateur et un `history` fourni par Gradio (liste de paires `[user, bot]`). Nous reconstruisons une `Conversation` à partir de cet historique, y ajoutons le message courant, appelons le pipeline, puis retournons la dernière réponse générée.

In [21]:
def mini_chatbot(message, history):
    if history is None:
        history = []
    # The conversational pipeline can handle the history directly
    # We need to format the history into a format the pipeline understands
    # The pipeline expects a list of dictionaries with 'user' and 'generated' keys
    formatted_history = []
    for user_msg, bot_msg in history:
      formatted_history.append({'user': user_msg, 'generated': bot_msg})

    # Pass the current message and formatted history to the chatbot
    response = chatbot(message, chat_history=formatted_history)

    # The response is a Conversation object, the latest generated response is the last one
    return response.generated_responses[-1] if response.generated_responses else ""

## Étape 4 — Construire l’interface Gradio

Nous utilisons `gr.ChatInterface` qui prend une fonction de dialogue et gère l’historique automatiquement. Le titre et la description aident à documenter l’interface.

In [22]:
demo_chatbot = gr.ChatInterface(
    fn=mini_chatbot,
    title="Chatbot Transformers — BlenderBot 400M distill",
    description="Discutez avec un modèle BlenderBot distillé. L'historique de la conversation est conservé pour fournir du contexte."
)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


## Étape 5 — Lancer l’application

Exécutez la cellule suivante pour démarrer l’interface. Une URL locale s’affiche. Ouvrez-la dans votre navigateur puis commencez à converser. Le premier échange peut être plus lent à cause du chargement des poids du modèle.

In [23]:
demo_chatbot.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Bonnes pratiques et extensions possibles

- Ajouter une limite de longueur de contexte pour éviter un historique trop volumineux.
- Forcer une langue en prétraitant l’entrée, ou détecter la langue automatiquement.
- Mettre en cache le pipeline ou charger le modèle sur GPU si disponible pour accélérer les réponses.
- En production, ajouter une modération de contenu et une journalisation des erreurs.

In [ ]:
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

MODEL_NAME = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(MODEL_NAME)
model = BlenderbotForConditionalGeneration.from_pretrained(MODEL_NAME)

def generate_response(user_input, chat_history):
    # Encode the conversation history and the new user input
    # The model expects the history to be in a specific format, usually alternating user and bot messages
    # We'll format the history as a single string
    history_string = ""
    for user_msg, bot_msg in chat_history:
        history_string += f"<s>{user_msg}</s>{bot_msg}</s>"
    history_string += f"<s>{user_input}</s>"

    # Tokenize the input string
    inputs = tokenizer([history_string], return_tensors='pt')

    # Generate the response
    reply_ids = model.generate(**inputs)

    # Decode the generated response
    response = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
    return response

# Example usage:
chat_history = []
user_input = "Hi, I am testing a BlenderBot chatbot. How are you today?"
response = generate_response(user_input, chat_history)
print(f"User: {user_input}")
print(f"Bot: {response}")
chat_history.append([user_input, response])

user_input = "What can you do? Give me a very short answer."
response = generate_response(user_input, chat_history)
print(f"User: {user_input}")
print(f"Bot: {response}")
chat_history.append([user_input, response])

print("\nFull chat history:", chat_history)